## Preamble

In [ ]:
# Allows for interactive shell - outputs all non variable statements
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# To plot pretty figures
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 1
%aimport keras_vgg16
from keras_vgg16 import *

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


## Project Setup

In [2]:
import os
import shutil
from glob import glob
np.random.seed(10)

current_dir = os.getcwd()
DATASET_DIR=os.path.join(current_dir, 'dataset')
CROSSVALID_DIR=os.path.join(DATASET_DIR, 'cross_valid')
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
TEST_DIR = os.path.join(DATASET_DIR, 'test')
CROSSVALID_DIR = os.path.join(DATASET_DIR, 'cross_valid')
SAMPLE_DIR = os.path.join(DATASET_DIR, 'sample')

SAMPLE_TRAIN_DIR=os.path.join(SAMPLE_DIR, 'train')
SAMPLE_CROSSVALID_DIR=os.path.join(SAMPLE_DIR, 'cross_valid')

WEIGHTS_DIR = os.path.join(current_dir, 'weights')

## Model preparation

### Finetune the keras model
* Pop the last layer, freeze all layers, add a softmax layer and update set of classes

In [3]:
vgg16 = KerasVgg16(WEIGHTS_DIR)
vgg16.create_model(learning_rate = 0.01, ttl_outputs=2)
vgg16.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Fit the keras model
1. Train the updated keras model

In [4]:
# 581 seconds per epoch when use_multiprocessing=False
# train_generator = vgg16.generator(TRAIN_DIR, 100, shuffle=False)
# valid_generator = vgg16.generator(CROSSVALID_DIR, 100, shuffle=False)
# vgg16.finetune(train_generator, valid_generator, epochs=3)

# use_multiprocessing=True - couldn't pickle..doesn't work with tensorflow?
# vgg16.finetune(train_generator, valid_generator, epochs=2, use_multiprocessing=True)

# bcolz approach
train_generator = vgg16.generator(TRAIN_DIR, 50, shuffle=False)
valid_generator = vgg16.generator(CROSSVALID_DIR, 50, shuffle=False)
vgg16.save_generator(train_generator, 'sample_train')
vgg16.save_generator(valid_generator, 'sample_valid')

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


MemoryError: 

In [ ]:
trn = vgg16.load_bcolz_generator('sample_train', vgg16.onehot(train_generator), 35)
valid = vgg16.load_bcolz_generator('sample_valid', vgg16.onehot(valid_generator), 35)

In [ ]:
vgg16.finetune(trn, valid, epochs=2)

### Save and load the model after couple of epochs

In [ ]:
# filename='3_epochs_finetune_96'
# vgg16.save_weights(filename)
# vgg16.load_weights(filename)

## Perform predictions

## Kaggle Submit

### Prepare csv file and Submit
